# Backbone

In [ ]:
import torch.nn as nn

class DownTransition(nn.Module):
    def __init__(self, in_ch, depth, act):
        super(DownTransition, self).__init__()
    pass

class UpTransition(nn.Module):
    def __init__(self, in_ch, out_ch, depth, act):
        super(UpTransition, self).__init__()
    pass

class UNet3D(nn.Module):
    def __init__(self, act='relu'):
        super(UNet3D, self).__init__()
        
        self.down_tr64 = DownTransition(1,0,act)
        self.down_tr128 = DownTransition(64,1,act)
        self.down_tr256 = DownTransition(128,2,act)
        self.down_tr512 = DownTransition(256,3,act)

        self.up_tr256 = UpTransition(512, 512,2,act)
        self.up_tr128 = UpTransition(256,256, 1,act)
        self.up_tr64 = UpTransition(128,128,0,act)
        # self.out_tr = OutputTransition(64, n_class)
    
    def forward(self, x):
        self.out64, self.skip_out64 = self.down_tr64(x)
        self.out128,self.skip_out128 = self.down_tr128(self.out64)
        self.out256,self.skip_out256 = self.down_tr256(self.out128)
        self.out512,self.skip_out512 = self.down_tr512(self.out256)  # 降采样后得到的高层次特征表示

        self.out_up_256 = self.up_tr256(self.out512,self.skip_out256)
        self.out_up_128 = self.up_tr128(self.out_up_256, self.skip_out128)
        self.out_up_64 = self.up_tr64(self.out_up_128, self.skip_out64)  # 上采样后供分类的低层次特征表示
        # self.out = self.out_tr(self.out_up_64)

        return self.out512, self.out_up_64

In [ ]:
import torch.nn as nn

class UM(nn.Module):
    def __init__(self):
        super(UM, self).__init__()
        self.backbone = UNet3D()  # 选择骨干网络
        pass

    def forward(self, x_in):
        dec4, x = self.backbone(x_in)  # 骨干网络返回两个结果

在 `Universal_model` 类中，`UNet3D` 网络通常会返回两个主要的输出，分别是 `out512` 和 `out_up_64`。这两个输出代表不同阶段的特征图，具体如下：

### 1. `out512`
- **描述**: `out512` 是从 `UNet3D` 的最后一层（通常是编码器的最后一层）得到的特征图，具有 512 个通道。
- **用途**: 
  - 这个输出包含了对输入数据的高层次、抽象的特征表示，通常用于进行进一步的分类或回归任务。
  - 在医学图像分割任务中，`out512` 会被用作后续处理的输入，可能会经过一些额外的卷积层或全局平均池化层，以提取更具语义的信息。

### 2. `out_up_64`
- **描述**: `out_up_64` 是从 `UNet3D` 的上采样阶段得到的特征图，通常具有 64 个通道。
- **用途**: 
  - 这个输出通常是经过解码器的上采样过程后得到的，包含了较低层次的细节信息。
  - 在分割任务中，`out_up_64` 可能用于与其他特征图（如 `out512`）进行拼接，帮助恢复图像细节，从而提高分割精度。

### 总结
- **`out512`**: 高层次的特征表示，通常用于分类或作为进一步处理的基础。
- **`out_up_64`**: 经过上采样的特征，包含细节信息，常用于精细化分割任务。

这两个输出结合使用，使得模型在处理医学图像时，既能捕获全局语义信息，又能保留局部细节，从而提高整体性能。

# UM forward

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UM(nn.Module):
    def __init__(self,img_size, in_ch, out_ch, backbone = 'Unet3', encoding = 'rand_embedding'):
        super(UM, self).__init__()
        
        # 获取骨干网络参数
        self.backbone_name = backbone
        if backbone == 'Unet':
            self.backbone = UNet3D()
            
            # 预处理层(骨干网络最终输出out,输入进分割头前)
            self.precls_conv = nn.Sequential(
                nn.GroupNorm(16, 64),
                nn.ReLU(inplace=True),
                nn.Conv3d(64,8, kernel_size=1, stride=1, padding=0),
            )
            
            # 全局平均池化GAP(应用于高层次特征)
            self.GAP = nn.Sequential(
                nn.GroupNorm(16,512),
                nn.ReLU(inplace=True),
                torch.nn.AdaptiveAvgPool3d((1,1,1)),
                nn.Conv3d(in_channels=1, out_channels=256, kernel_size=1, stride=1, padding=0),
            )
        
        pass
        
        # 获取权重和偏置数
        weight_nums, bias_nums = [], []
        pass
        self.weight_nums = weight_nums
        self.bias_nums = bias_nums
        
        # 控制器(用于生成动态模型参数)
        self.controller = nn.Conv3d(256+256, sum(weight_nums+bias_nums), kernel_size=1, stride=1, padding=0)
        
        # 获取编码方式参数
        self.encoding = encoding
        if self.encoding == 'rand_embedding':
            self.organ_embedding = nn.Embedding(out_ch, 256)
        elif self.encoding == 'word_embedding':
            self.register_buffer('organ_embedding', torch.randn(out_ch, 512))
            self.text_to_vision = nn.Linear(512, 256)
        
        # 设置分类数
        self.class_num = out_ch
    
    def load_params(self):
        pass
    
    def encoding_task(self):
        pass
    
        
    def parse_dynamic_params(self, params, channels, weight_nums, bias_nums):
        pass
        return _, _
    
    def heads_forward(self, features, weights, biases, num_insts):
        return _
    
    def forward(self, x_in):
        # 获取backbone的两个输出
        dec4, out = self.backbone(x_in)  # 返回:降采样后高层次特征dec4,上采样后低层次特征out
        
        # 选择嵌入方式
        if self.encoding == 'rand_embedding':
            task_encoding = self.organ_embedding.weight.unsqueeze(2).unsqueeze(2).unsqueeze(2)
        elif self.encoding == 'word_embedding':
            task_encoding = F.relu(self.text_to_vision(self.organ_embedding))
            task_encoding = task_encoding.unsqueeze(2).unsqueeze(2).unsqueeze(2)
        # task_encoding torch.Size([31,256,1,1,1])
        
        # 全局平均池化GAP
        x_feat = self.GAP(dec4)   # torch.Size([batch_size,channels,1,1,1])
        b = x_feat.shape[0]  # 批次大小
        
        # 循环处理每个实例
        
        # 初始化logits数组,用于存储每个实例的输出logits
        logits_array = []
        
        # 遍历每个实例
        for i in range(b):
            # 将当前实例的特征x_feat[i](经过unsqueeze增加维度后)和任务编码拼接,形成条件输出
            x_cond = torch.cat([x_feat[i].unsqueeze(0).repeat(self.class_num,1,1,1,1), task_encoding], 1)
            
            # 生成动态参数
            params = self.controller(x_cond)  # 通过控制器生成动态参数
            params.squeeze_(-1).squeeze_(-1).squeeze_(-1)  # 删除后三维,降低维度
            
            # 预处理骨干网络输出out,供分割头使用
            head_inputs = self.precls_conv(out[i].unsqueeze(0))
            head_inputs = head_inputs.repeat(self.class_num,1,1,1,1)  # 重复class_num次,得到适用于每个类的输出
            
            # 获取尺寸,重塑以进行卷积
            N, _, D, H, W = head_inputs.size()
            head_inputs = head_inputs.reshape(1,-1,D,H,W)
            # print(head_inputs.shape, params.shape)
            
            # 解析动态参数
            weights, biases = self.parse_dynamic_params(params, 0, self.weight_nums, self.bias_nums)
            
            # 执行头部前向计算
            logits = self.heads_forward(head_inputs, weights, biases, N)  # 得到当前实例的输出
            
            # 存储输出
            logits_array.append(logits.reshape(1,-1,D,H,W))
            
        # 合并输出
        out = torch.cat(logits_array, dim=0)
        
        return out   

# todo
1. 选择编码方式
    1. nn.Embedding
    2. register_buffer 
2. logits数组
3. 权重和偏置数
4. 生成动态参数
5. 参数控制器controller
6. heads_forward